## Imports

In [3]:
# standard lib
import os, pwd, sys, json, yaml, atexit, tempfile, inspect

# for data-science
import pandas as pd, numpy as np, quadfeather
from pyarrow import feather

# for plotting
import matplotlib as mpl, matplotlib.pyplot as plt, seaborn as sns

# for cellular-data
import scprep, scanpy as sc, anndata as ad

## Setup

In [2]:
SEED = 3
np.random.seed(SEED)

In [4]:
adata = ad.read_h5ad(os.path.join(os.path.expanduser('~/Downloads'), 'adata_s1.h5'))

In [133]:
adata.var

,ensembl_id,gene_symbol,feature_type,mito,ribo,n_cells_by_counts,mean_counts,log1p_mean_counts,pct_dropout_by_counts,total_counts,log1p_total_counts,n_cells,highly_variable_WT1,highly_variable_WT2,highly_variable_KO1,highly_variable_KO3,highly_variable
mir1302-2hg,ENSG00000243485,MIR1302-2HG,Gene Expression,False,False,11,0.000267,0.000267,99.973335,11.0,2.484907,11,False,False,False,False,False
ensg00000238009,ENSG00000238009,ENSG00000238009,Gene Expression,False,False,52,0.001285,0.001284,99.873949,53.0,3.988984,52,False,False,False,False,False
ensg00000239945,ENSG00000239945,ENSG00000239945,Gene Expression,False,False,6,0.000145,0.000145,99.985456,6.0,1.945910,6,False,False,False,False,False
ensg00000241860,ENSG00000241860,ENSG00000241860,Gene Expression,False,False,93,0.002254,0.002252,99.774562,93.0,4.543295,93,False,False,False,False,False
linc01409,ENSG00000237491,LINC01409,Gene Expression,False,False,252,0.006133,0.006114,99.389135,253.0,5.537334,252,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ensg00000276345,ENSG00000276345,ENSG00000276345,Gene Expression,False,False,5944,0.165103,0.152810,85.591351,6811.0,8.826441,5944,False,False,False,True,True
ensg00000276760,ENSG00000276760,ENSG00000276760,Gene Expression,False,False,7,0.000170,0.000170,99.983032,7.0,2.079442,7,False,False,False,False,False
ensg00000273554,ENSG00000273554,ENSG00000273554,Gene Expression,False,False,110,0.002715,0.002711,99.733353,112.0,4.727388,110,False,False,False,False,False
ensg00000278817,ENSG00000278817,ENSG00000278817,Gene Expression,False,False,2092,0.052772,0.051427,94.928854,2177.0,7.686162,2092,False,False,False,False,False


In [25]:
EMB = 'X_phate'
COUNT_LAYER = 'X_magic'

X_cnt = adata.layers.get(COUNT_LAYER)
X_emb = adata.obsm[EMB]


barcodes = adata.obs.barcodes
condition = adata.obs.batch

genes = adata.var.gene_symbol.values

emb_name = EMB.split('_')[1].upper()
emb_cols = [f'{emb_name}_{i+1}' for i in range(X_emb.shape[1])]

In [50]:
# get HVGs
high_variable = pd.Series(adata.var[adata.var.highly_variable].gene_symbol.values.to_numpy(), name="hvg_genes")
# find their var_name
vnames = adata.var_names[adata.var_names.isin(high_variable.map(lambda e: e.lower()))]
# get the corresponding indices
vidxs = [adata.var_names.tolist().index(vname) for vname in vnames]
# make DF (not some genes have same gene symbol so we lose 2 in this instance)
df_hvg = pd.DataFrame( 
    adata[:, vidxs].layers.get(COUNT_LAYER).toarray(), index=barcodes, 
    columns=pd.Series(vnames, name=None)
)
df_hvg.head()

,samd11,hes4,isg15,ensg00000242590,pusl1,aurkaip1,ccnl2,mrpl20-as1,mrpl20,vwa1,...,sry,rps4y1,zfy,tbl1y,usp9y,ddx3y,tmsb4y,ttty14,ensg00000273748,ensg00000276345
barcodes,,,,,,,,,,,,,,,,,,,,,
AAAAAAAAACCGCAAA,0.002811,0.160271,0.419841,0.024929,0.144510,0.934147,0.049386,0.099603,1.215815,0.021958,...,0.034613,1.454904,0.020024,0.065525,0.048129,0.060256,0.135738,0.741320,0.037747,0.215293
AAAAAAACAGCTACAC,0.004659,0.142076,0.364149,0.040240,0.118234,0.919216,0.064004,0.091365,1.193966,0.020583,...,0.046903,1.412669,0.025799,0.084932,0.053276,0.061857,0.100414,0.647127,0.035790,0.198256
AAAAAAAGAGAAAGAT,0.018177,0.178742,0.267882,0.033760,0.111332,0.898725,0.062531,0.106328,1.060256,0.020664,...,0.042020,1.428924,0.021415,0.042015,0.061298,0.051220,0.074577,0.531677,0.034472,0.180006
AAAAAAAGAGTGCCTC,0.003250,0.138905,0.486379,0.028987,0.148792,0.901993,0.055240,0.089119,1.189651,0.012499,...,0.027108,1.439636,0.019846,0.071122,0.048794,0.055076,0.142634,0.743455,0.053719,0.199488
AAAAAAAGCCGTCCCA,0.016494,0.142930,0.292623,0.030721,0.112903,0.922066,0.058253,0.117850,1.092841,0.020914,...,0.045965,1.417680,0.026653,0.056081,0.063033,0.063136,0.098628,0.618320,0.034030,0.210653


In [49]:
df_emb = pd.DataFrame(
    X_emb,  index=barcodes, columns=emb_cols
)
df_emb.loc[:, 'condition'] = condition.values
df_emb.head()

,PHATE_1,PHATE_2,PHATE_3,condition
barcodes,,,,
AAAAAAAAACCGCAAA,-0.021562,-0.000908,0.000604,WT1
AAAAAAACAGCTACAC,-0.025020,-0.000228,0.001007,WT1
AAAAAAAGAGAAAGAT,-0.030869,0.002139,0.002990,WT1
AAAAAAAGAGTGCCTC,-0.023341,-0.001362,0.000606,WT1
AAAAAAAGCCGTCCCA,-0.028022,0.001227,0.001963,WT1


In [51]:
df_emb.to_parquet(os.path.join('~/Downloads/embedding.parquet'))

In [63]:
df_emb.rename({
    'PHATE_1': 'x',
    'PHATE_2': 'y',
    'PHATE_3': 'z',
}, axis=1).to_parquet(os.path.join('~/Downloads/embedding.parquet'))

In [52]:
df_hvg.to_parquet(os.path.join('~/Downloads/hvg_genes.parquet'))

In [54]:
TILE_SIZE = 5000

In [55]:

# full path to this notebook
FILE = os.path.abspath('')

# the sveltekit project you might be working on / want to deploy
SVELTEKIT_DIR = os.path.join(FILE, '..')

# the static assets directory of the sveltekit project where files are hosted
STATIC_ASSETS_DIR = os.path.join(SVELTEKIT_DIR, 'static')

# we are assuming that you might have multiple datasets you want to host / switch between
DATASETS_DIR = os.path.join(STATIC_ASSETS_DIR, 'datasets')

# this is where we are going to store our dataset
DATASET_NAME = 'seo'
TARGET_DIR = os.path.join(DATASETS_DIR, DATASET_NAME)

In [64]:
!quadfeather --files {os.path.join('~/Downloads/embedding.parquet')} \
        --tile_size {TILE_SIZE} \
        --destination {os.path.join(TARGET_DIR, 'tiles')}

In [ ]:
!rm -rf {os.path.join(TMP_DIR, '_deepscatter_tmp')}

### 2) make single file

Create a single file that contains all the data you want to add, but none of the data that’s already there except for your unique id field (`label` in this case). 

NOTE: `label` must be the same name and data type as in your primary file.

In [125]:
df_join = df_emb.reset_index().join(df_hvg.reset_index(), how='left', rsuffix='_BAD').dropna()

In [126]:
df_join = df_join.set_index('barcodes').drop(columns=['barcodes_BAD'] + df_emb.columns.values.tolist())

NOTE: **All** columns that you want to show up in the data should ideally be `float32()` type, although doubles might not be the end of the world.

In [127]:
# df_join = df_join.astype('float32')
df_join.head()

,samd11,hes4,isg15,ensg00000242590,pusl1,aurkaip1,ccnl2,mrpl20-as1,mrpl20,vwa1,...,sry,rps4y1,zfy,tbl1y,usp9y,ddx3y,tmsb4y,ttty14,ensg00000273748,ensg00000276345
barcodes,,,,,,,,,,,,,,,,,,,,,
AAAAAAAAACCGCAAA,0.002811,0.160271,0.419841,0.024929,0.144510,0.934147,0.049386,0.099603,1.215815,0.021958,...,0.034613,1.454904,0.020024,0.065525,0.048129,0.060256,0.135738,0.741320,0.037747,0.215293
AAAAAAACAGCTACAC,0.004659,0.142076,0.364149,0.040240,0.118234,0.919216,0.064004,0.091365,1.193966,0.020583,...,0.046903,1.412669,0.025799,0.084932,0.053276,0.061857,0.100414,0.647127,0.035790,0.198256
AAAAAAAGAGAAAGAT,0.018177,0.178742,0.267882,0.033760,0.111332,0.898725,0.062531,0.106328,1.060256,0.020664,...,0.042020,1.428924,0.021415,0.042015,0.061298,0.051220,0.074577,0.531677,0.034472,0.180006
AAAAAAAGAGTGCCTC,0.003250,0.138905,0.486379,0.028987,0.148792,0.901993,0.055240,0.089119,1.189651,0.012499,...,0.027108,1.439636,0.019846,0.071122,0.048794,0.055076,0.142634,0.743455,0.053719,0.199488
AAAAAAAGCCGTCCCA,0.016494,0.142930,0.292623,0.030721,0.112903,0.922066,0.058253,0.117850,1.092841,0.020914,...,0.045965,1.417680,0.026653,0.056081,0.063033,0.063136,0.098628,0.618320,0.034030,0.210653


The file must be a [feather file][feather file], not parquet. 

```python 
from pyarrow import feather; 

# if converting from parquet
feather.write_feather(parquet.read_table('fin.parquet), 'fout.feather))

# if converting pandas
feather.write_feather(df, 'fout.feather')
```

[feather file]: https://arrow.apache.org/docs/python/feather.html

In [128]:
feather.write_feather(df_join, os.path.join('~/Downloads/sidecar.feather'))

In [129]:
df_feather = feather.read_feather(os.path.join('~/Downloads/sidecar.feather'))

In [130]:
LABEL_NAME = 'barcodes'

### 3) run `add_sidecars.py`

In [131]:
!python3 add_sidecars.py --tileset {os.path.join(TARGET_DIR, 'tiles')}\
                         --sidecar {os.path.join('~/Downloads/sidecar.feather')} --key {LABEL_NAME};

Traceback (most recent call last):
  File "/Users/solst/Projects/featherplot/nbs/add_sidecars.py", line 102, in <module>
    add_sidecars_cli()
  File "/Users/solst/Projects/featherplot/nbs/add_sidecars.py", line 99, in add_sidecars_cli
    tileset.add_sidecars(args.sidecar, args.key)
  File "/Users/solst/Projects/featherplot/nbs/add_sidecars.py", line 67, in add_sidecars
    all_ixes = pc.index_in(master_lookup_tb[key], sidecar_table[key])
  File "/Users/solst/mambaforge/envs/quadfeather/lib/python3.10/site-packages/pyarrow/compute.py", line 259, in wrapper
    return func.call(args, options, memory_pool)
  File "pyarrow/_compute.pyx", line 367, in pyarrow._compute.Function.call
  File "pyarrow/error.pxi", line 144, in pyarrow.lib.pyarrow_internal_check_status
  File "pyarrow/error.pxi", line 100, in pyarrow.lib.check_status
pyarrow.lib.ArrowInvalid: Array type didn't match type of values set: string vs dictionary<values=string, indices=int16, ordered=0>


In [ ]:
file_shapes = dict()
bad_files = set()
missing_per_files = dict()
total_files = 0
walking_sum = 0
for root, dirs, files in os.walk(os.path.join(TARGET_DIR, 'tiles')):
    filtered = filter(lambda f: f.endswith('.feather') and len(f.split('.')) == 2, files)
    for f in filtered:
        cur_file = os.path.join(root, f)
        df_shape = feather.read_feather(cur_file).shape
        file_shapes[cur_file] = df_shape
        walking_sum += df_shape[0]
        total_files += 1
        if df_shape[0] < TILE_SIZE:
            bad_files.add(cur_file)
            missing_per_files[cur_file] = abs(df_shape[0] - TILE_SIZE)

print(f'''
Summary:
--------
Total rows: {walking_sum}

Total files: {total_files}

    • N bad files: {len(bad_files)}
    • Missing rows per file: {list(missing_per_files.values())}

Expected rows: {df_all.shape[0]}
   • Points csv shape {pd.read_csv(csv_points.name).shape}    
   • Sidecar csv shape {pd.read_csv(csv_points.name).shape}    
   • Sidecar feather file shape: {df_feather.shape}
''')            


Summary:
--------
Total rows: 8590

Total files: 16

    • N bad files: 10
    • Missing rows per file: [317, 807, 958, 983, 997, 107, 931, 478, 984, 848]

Expected rows: 20000
   • Points csv shape (20000, 4)    
   • Sidecar csv shape (20000, 4)    
   • Sidecar feather file shape: (20000, 5)



## Meta Data

In [ ]:
meta = dict(
    seed=SEED, n_points=N_POINTS, tile_size=TILE_SIZE, 
    dataset_name=DATASET_NAME, label_name=LABEL_NAME,
    
    # NOTE: since all these direcetories are relative to the static assets directory
    #       we can use the relative path to the static assets directory instead of the wrangling
    #       we did above.
    target_dir=TARGET_DIR.replace(STATIC_ASSETS_DIR, ''), 
    tiles_dir=os.path.join(TARGET_DIR, 'tiles').replace(STATIC_ASSETS_DIR, ''),

    embedding_columns=df_p.columns.values.tolist(),
    sidecar_columns=df_s.columns.values.tolist(),
    column_metadata=column_meta,
)

In [ ]:
with open(os.path.join(TARGET_DIR, 'meta.yml'), 'w') as f:
    f.write(yaml.dump(meta))

## Cleanup

NOTE: these files will automatically be deleted when the kernel stops, but we delete them here for good practice

In [ ]:
csv_points.close()
csv_sidecar.close()
feather_sidecar.close()